In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import NarrowbandMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
import torchsig.transforms.functional as F

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def generate_qpsk_signal(num_iq_samples: int = 128, scale: float = 1.0) -> Signal:
    """Generate a scaled, high SNR baseband QPSK Signal.

        Args:
        num_iq_samples (int, optional): Length of sample. Defaults to 10.
        scale (int, optional): scale normalized signal data. Defaults to 1.0.

        Returns:
            signal: generated Signal 

    """
    sample_rate = 10e6
    md = NarrowbandMetadata(
        num_iq_samples_dataset = num_iq_samples,
        fft_size = 4,
        impairment_level = 0,
        sample_rate = sample_rate,
        num_signals_min = 1,
        num_signals_distribution = [1.0],
        snr_db_min = 100.0,
        snr_db_max = 100.0,
        signal_duration_min = 1.00*num_iq_samples/sample_rate,
        signal_duration_max = 1.00*num_iq_samples/sample_rate,
        signal_bandwidth_min = sample_rate/4,
        signal_bandwidth_max = sample_rate/4,
        signal_center_freq_min = 0.0,
        signal_center_freq_max = 0.0,         
        class_list = ['qpsk'],
        class_distribution = [1.0],
        seed = 42
    )

    builder = ConstellationSignalBuilder(
        dataset_metadata = md, 
        class_name = 'qpsk',
        seed = 42
    )
    signal = builder.build()

    # normalize, then scale data   
    signal.data = F.normalize(
        data = signal.data,
        norm_order = 2,
        flatten = False
    )
    signal.data = np.multiply(signal.data, scale)

    return signal

In [ ]:
# test data
N = 1024
qpsk_data = generate_qpsk_signal(num_iq_samples = N, scale = 1.0).data

# add in a tiny amount of noise to avoid log10(0)
complex_noise = np.sqrt(1e-6)*(np.random.normal(0,1,N) + 1j*np.random.normal(0,1,N))
qpsk_data += complex_noise

impaired_qpsk_data = F.coarse_gain_change(
    data = qpsk_data,
    gain_change_db=20,
    start_idx = N//2
)

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(2,1,1)
ax.plot(20*np.log10(np.abs(qpsk_data)),label='Input')
ax.plot(20*np.log10(np.abs(impaired_qpsk_data)),label='With Gain Change')
ax.grid()
ax.set_title('Constellation Plot')
ax.set_xlabel('Time Index $n$')
ax.set_ylabel('Magnitude (dB)')
ax.legend(fontsize='large', loc='upper left');

plt.show()